# Setup

In [1]:
!pwd

/home/umni2/a/umnilab/users/verma99/mk/spr_4711/code


In [2]:
from mobilkit.umni import *
from project import *

In [3]:
from dash import Dash, dcc, html, Input, Output
import dash_bootstrap_components as dbc
import plotly.express as px
# import plotly.graph_objects as go

In [4]:
app = Dash(__name__)
app.title = 'Indiana Transport Equity Atlas'

In [5]:
app.layout = html.Div([
    html.H1('My own web application dashboards with Dash', style=D(
        textAlign='left', fontFamily='Noto Sans', color='purple')),
    dcc.RadioItems(id='scale-selector', value='County', inline=True,
                   style=D(width='40%', fontFamily='Arial', size='14px'),
                   options=[D(label=x, value=x) for x in SCALES]),
    html.Div(id='output-container', children=[]),
    html.Br(),
    dcc.Graph(id='map-main')
])

In [6]:
zones = gpd.read_parquet('../data/zones/zones_2010.parquet').set_crs(CRS_DEG).disp()

6,417 rows x 6 cols; Memory: 1.4 MiB; CRS: EPSG:4326


,geoid,name,aland,awater,geometry,scale
,<object>,<object>,<int64>,<int64>,<geometry>,<object>
0,18083,Knox,1336514262,20755159,"POLYGON ((-87.506221 38.733533, -87.506172 38....",COUNTY


In [ ]:
ejs = pd.read_parquet('../data/ejs/ejs.parquet').disp()

356,286 rows x 7 cols; Memory: 5.8 MiB


,scale,is_ses,is_derived,is_pctile,variable,geoid,value
,<category>,<bool>,<bool>,<bool>,<category>,<category>,<float64>
0,TRACT,True,False,False,Total: Population,18001030100,4826.0


In [8]:
@app.callback(
    Output('map-main', 'figure'),
    Input('scale-selector', 'value'))
def plot_map(scale, zones=zones):
    df = zones.query(f'scale=="{scale.upper()}"')
    df = df.assign(area_sqmi=df.aland / 2.59e+6)
    # fig = px.choropleth(
    #     df.assign(area_sqmi=df.aland / 2.59e+6),
    #     geojson=zjson[scale.upper()],
    #     template='plotly_dark', rendermode='webgl',
    #     locations='geoid', color='area_sqmi')
    # fig.update_geos(scope='usa', fitbounds='geojson', visible=True)
    fig = px.choropleth(
        df,
        geojson=df.geometry,
        locations=df.geoid,
        color='area_sqmi',
        # projection='transverse mercator',
        template='plotly',
        # template='plotly_dark',
    )
    fig.update_geos(scope='usa', fitbounds='geojson', visible=True)
    fig.update_layout(height=1000, width=800, margin=D(l=0, r=0, b=0, t=0))
    return fig

In [9]:
app.run(debug=True, port=8005, threaded=True, jupyter_mode='external')

Dash app running on http://127.0.0.1:8005/
